<a href="https://colab.research.google.com/github/yodaboop/Prompt-Engineer/blob/main/Customer_Support_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code Generation with ReACT Prompting Using Gemini

**Prompt:**

Your task is to build a CustomerSupportAgent that has the script using API key and the system prompt like "You are 'SupportBot,' a friendly and efficient AI assistant for a major internet service provider. A user's first message in a new chat is: "My internet is completely down. I can't get any work done. Help!"
Your goal is to:
Acknowledge their frustration.
Ask the single most common diagnostic question (rebooting the router).

I need to run it from a collab file.

In [1]:
import os
import requests
import time
import json
from google.colab import userdata

# --- 1. CONFIGURATION ---

# IMPORTANT: This script requires a Gemini API key.
# Set it as an environment variable in your system or as a secret in Colab.
# Example in terminal: export GEMINI_API_KEY="YOUR_API_KEY_HERE"
# API_KEY = os.environ.get("GEMINI_API_KEY")
API_KEY = userdata.get('GEMINI_API_KEY')

API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent?key={API_KEY}"

# --- 2. SYSTEM PROMPTS (THE "BRAIN" OF THE AGENT) ---
# This dictionary controls the AI's behavior at each step.

PROMPTS = {
    'START': """You are 'SupportBot,' an AI assistant for a major internet service provider. Your persona is friendly, efficient, and empathetic.
        Your goal is to generate the *very first* response to a user's problem.
        RULES:
        1. Your response must be a single, concise paragraph.
        2. You MUST start by acknowledging the user's frustration.
        3. Your response MUST end by asking the single most common diagnostic question: whether the user has already tried rebooting their modem and router.""",

    'ASK_LIGHTS': """You are 'SupportBot.' The user just confirmed they *HAVE* already tried rebooting, but it didn't work.
        RULES:
        1. Acknowledge their effort (e.g., "Thanks for confirming that.").
        2. Your next goal is to ask for the *second* most common diagnostic: the indicator lights on the router.
        3. Ask them to look at the device and describe the 'Internet' or 'WAN' light (its color and whether it's blinking or solid).""",

    'GUIDE_REBOOT': """You are 'SupportBot.' The user just confirmed they have *NOT* tried rebooting yet.
        RULES:
        1. Be friendly and reassuring (e.g., "No problem, let's walk through that...").
        2. Provide simple, clear, step-by-step instructions for a full power cycle:
           (1) Unplug power, (2) Wait 60 seconds, (3) Plug back in.
        3. Explain that it might take 2-3 minutes to reconnect.
        4. Ask them to let you know if the connection comes back.""",

    'ESCALATE': """You are 'SupportBot.' The user's diagnostic has failed (e.g., they reported a 'red light'). This is not solvable by the user.
        RULES:
        1. Be empathetic and thank them for checking.
        2. Clearly state that this indicates a network-side problem that requires a technician.
        3. Your goal is to *escalate*.
        4. To do this, you MUST ask for the full name and best contact number for the account.""",

    'RESOLVED': """You are 'SupportBot.' The user has indicated the problem is fixed (e.g., 'it works now' or 'green light').
        RULES:
        1. Express happiness or relief (e.g., "That's great news!").
        2. Polite ly close the conversation and invite them to reach out again if they need more help.""",

    'GENERAL': """You are 'SupportBot.' The user is saying something that doesn't fit the flow (e.g., "how much is my bill?").
        RULES:
        1. Gently guide them back to the troubleshooting task.
        2. Say, "I can only help with technical troubleshooting right now. Can we continue with the current issue?"""
}

# --- 3. CORE LOGIC ---

def get_next_system_prompt(history):
    """
    This is the "Prompt Chaining" logic.
    It analyzes the conversation and decides which system prompt to use *next*.
    """
    if not history:
        return PROMPTS['START']

    try:
        # Get the last *user* message
        last_user_message = next(msg['parts'][0]['text'] for msg in reversed(history) if msg['role'] == 'user').lower()

        # Get the last *bot* message
        last_bot_message = next(msg['parts'][0]['text'] for msg in reversed(history) if msg['role'] == 'model').lower()

        # State 1: After bot asked to reboot...
        if 'rebooting' in last_bot_message:
            if 'yes' in last_user_message or 'i did' in last_user_message:
                return PROMPTS['ASK_LIGHTS']
            if 'no' in last_user_message or 'i have not' in last_user_message:
                return PROMPTS['GUIDE_REBOOT']

        # State 2: After bot asked about lights...
        if 'light' in last_bot_message:
            if 'red' in last_user_message or 'orange' in last_user_message:
                return PROMPTS['ESCALATE']
            if 'green' in last_user_message or 'blue' in last_user_message:
                return PROMPTS['RESOLVED']

        # State 3: After bot guided a reboot...
        if 'plug back in' in last_bot_message:
            if 'it works' in last_user_message or 'fixed' in last_user_message:
                return PROMPTS['RESOLVED']
            if 'still not working' in last_user_message or 'no luck' in last_user_message:
                # Chain to the next step: ask about lights
                return PROMPTS['ASK_LIGHTS']

    except StopIteration:
        # This can happen if there's no user or model message found yet
        pass

    # Fallback for general conversation or if logic doesn't match
    return PROMPTS['GENERAL']

def call_gemini_api(system_prompt, history):
    """
    Calls the Gemini API with exponential backoff.
    """
    payload = {
        "systemInstruction": {
            "parts": [{"text": system_prompt}]
        },
        "contents": history
    }

    retries = 3
    delay = 1  # 1 second

    for i in range(retries):
        try:
            response = requests.post(API_URL, json=payload, headers={'Content-Type': 'application/json'})

            if response.status_code == 200:
                result = response.json()

                # Check for safety ratings or blocked content
                if result.get('candidates') and len(result['candidates']) > 0:
                    candidate = result['candidates'][0]
                    if candidate.get('finishReason') == "SAFETY":
                        return "I'm sorry, I can't respond to that request due to safety policies."
                    return candidate['content']['parts'][0]['text']
                elif result.get('promptFeedback') and result['promptFeedback'].get('blockReason'):
                    return "My apologies, I am unable to process that request. (Prompt blocked)"
                else:
                   raise Exception("No valid candidate in response.")


            elif response.status_code == 429:  # Throttling
                print(f"API Throttled. Retrying in {delay}s...")
                time.sleep(delay)
                delay *= 2  # Exponential backoff
            else:
                # Other HTTP errors
                raise Exception(f"API Error: {response.status_code} - {response.text}")

        except requests.exceptions.RequestException as e:
            print(f"Attempt {i + 1} failed: {e}")
            if i == retries - 1:
                raise  # Rethrow last error
            time.sleep(delay)
            delay *= 2

    raise Exception("API request failed after all retries.")

def main():
    """
    Main function to run the chat loop.
    """
    if not API_KEY:
        print("="*50)
        print("ERROR: GEMINI_API_KEY not found in Colab secrets.")
        print("Please add the API key to Colab secrets with the name GEMINI_API_KEY.")
        print("="*50)
        return

    conversation_history = []

    # Start the conversation
    print("\nBot: Hello! I'm SupportBot. How can I help you with your internet today?")

    try:
        # Get the user's first problem description
        user_text = input("You: ")
        conversation_history.append({"role": "user", "parts": [{"text": user_text}]})

        # Get the first system prompt (PROMPTS['START'])
        system_prompt = get_next_system_prompt(conversation_history)

        # Get the first AI response
        bot_text = call_gemini_api(system_prompt, conversation_history)
        conversation_history.append({"role": "model", "parts": [{"text": bot_text}]})
        print(f"\nBot: {bot_text}")

        # Main conversation loop
        while True:
            user_text = input("You: ")
            if user_text.lower() in ['exit', 'quit']:
                print("\nBot: Thanks for chatting. Have a great day!")
                break

            conversation_history.append({"role": "user", "parts": [{"text": user_text}]})
            system_prompt = get_next_system_prompt(conversation_history)
            bot_text = call_gemini_api(system_prompt, conversation_history)
            conversation_history.append({"role": "model", "parts": [{"text": bot_text}]})
            print(f"\nBot: {bot_text}")

    except EOFError:
        print("\nBot: Chat ended. Goodbye!")
    except KeyboardInterrupt:
        print("\nBot: Chat interrupted. Goodbye!")
    except Exception as e:
        print(f"\nAn error occurred: {e}")

if __name__ == "__main__":
    main()


Bot: Hello! I'm SupportBot. How can I help you with your internet today?
You: I can't access to my SJSU one account

Bot: I understand you're having trouble accessing your SJSU One account.

To start troubleshooting, could you tell me what happens when you try to log in? Do you see an error message, or does the page just reload?
You: AWS not available

Bot: I appreciate you bringing that up, but the current issue we are addressing is your SJSU One account access.

I can only help with technical troubleshooting right now. Can we continue with the current issue?

When you tried to log into SJSU One, what specific error message did you receive?
You: no

Bot: I understand you may be frustrated, but I need specific details to help you resolve the login issue.

I can only help with technical troubleshooting right now. Can we continue with the current issue of accessing your SJSU One account?

To move forward, please describe the exact message or behavior you see when you try to log in.

Bot